### Autenticación
Primero autenticamos e inicializamos gee

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1ARtbsJqSWJDUkopJPSpXz6iHNfa8c-AtmpoDUv6tbfMqXCEcE_5NY2i3DrE

Successfully saved authorization token.


Vamos a utilizar el shp de Buenos Aires cargado en el práctico:

In [2]:
buenos_aires = ee.FeatureCollection('users/adelarja/Hidro2022/Buenos_Aires')

Para comparar con los datos obtenidos en el práctico, nos traemos las imágenes de agosto 2017.

In [3]:
sentinel_1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate('2017-08-01', '2017-08-31')

Creamos una función de clipping

In [5]:
def clipper(shape, image_set):
    """Clip the image set using the shape geometry."""
    return image_set.clip(shape)

Aplicamos el clipping a nuestro set

In [8]:
sentinel_1 = sentinel_1.map(lambda x: clipper(buenos_aires, x))

Nos quedamos con la polarización VV y VH, tomada por IW.

In [23]:
s1_vv_vh_iw = sentinel_1.filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
).filter(
    ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')
).filter(ee.Filter.eq('instrumentMode', 'IW'))

Ahora filtramos orbita ascendente y descendente en distintas colecciones

In [25]:
s1_vv_vh_iw_asc = s1_vv_vh_iw.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
s1_vv_vh_iw_desc = s1_vv_vh_iw.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

Nos quedamos con una imagen ascendente y otra descendente

In [47]:
asc = s1_vv_vh_iw_asc.first()
desc = s1_vv_vh_iw_desc.first()

Ahora nos quedamos con vv asc y desc y vh asc y desc

In [48]:
vv_asc = asc.select('VV')
vv_desc = desc.select('VV')

vh_asc = asc.select('VH')
vh_desc = desc.select('VH')

Configuramos folium para plotear la data

In [35]:
import folium

lat, lon = -30.749311, -64.057867

my_map = folium.Map(location=[lat, lon], zoom_start=22)

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [49]:
my_map.add_ee_layer(vv_asc, {'min': -12, 'max': -4}, "Ascendente")
my_map.add_ee_layer(buenos_aires, {}, "BsAs")
my_map